In [6]:
from tensorflow.keras.applications import VGG19
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [4]:
base_model = VGG19(weights='imagenet', include_top=False, input_shape=(300, 300, 3))

#custom layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(4, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=predictions)
for layer in base_model.layers:
    layer.trainable = False

80134624/80134624 [==============================] - 7s 0us/step


In [5]:
model.compile(optimizer=Adam(lr=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])


c:\Users\shady\anaconda3\envs\COMP9517\lib\site-packages\keras\optimizers\optimizer_v2\adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [8]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    #shear_range=0.2,
    #zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # if you want to split your dataset into training and validation
)

train_generator = train_datagen.flow_from_directory(
    '',
    target_size=(300, 300),
    batch_size=32,
    class_mode='categorical',
    subset='training'  # if using validation_split
)

validation_generator = train_datagen.flow_from_directory(
    '../elpv-dataset/images/.',
    target_size=(300, 300),
    batch_size=32,
    class_mode='categorical',
    subset='validation'  # if using validation_split
)



FileNotFoundError: [WinError 3] The system cannot find the path specified: ''

In [ ]:
model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // train_generator.batch_size,
    epochs=10,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // validation_generator.batch_size
)

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_directory(
    'path_to_test_directory',
    target_size=(300, 300),
    batch_size=32,
    class_mode='categorical'
)

evaluation = model.evaluate(test_generator)
print(f'Accuracy: {evaluation[1]*100:.2f}%')